<a href="https://colab.research.google.com/github/BecomeAllan/Bert_automation_SLR/blob/main/Utils/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies required


In [1]:
import ipywidgets as widgets
from IPython.display import HTML, display, clear_output, Markdown

from scipy import stats
import matplotlib.patches as mpatches

from ipywidgets import GridspecLayout
from scipy.stats import poisson

import pandas as pd
import numpy as np

import string

from itertools import chain
from google.colab import auth
import gspread
from gspread import Cell
import gspread_dataframe
from google.auth import default



In [19]:
#@title Judger()

class Judger:
    def __init__(self, data,sheet=None, researcher= 'researcher',
                 configure_text=True, random = True):
        self.data = data.copy()
        self.sheet = sheet
        self.cells = []
        self.researcher = researcher
        self.selected_indx = []
        self.index = 0

        compare_data = data.copy()
        
        compare_data = compare_data.replace({
                  'Included': 1,
                  'Not Included': -1,
                  '': np.nan
                  })
        
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        compare_data = compare_data.select_dtypes(include=numerics)


        # Density distribution to sample for labeling
        density = poisson.pmf(range(len(self.data)), mu=10)
        density[density == 0] = min(density[density != 0])

        # Randomizing indexes
        if random:
          self.shuffle = np.random.choice(
              range(len(data)), p=density, replace=False, size=len(data)
              )
        else:
          # Procura se tem a coluna do pesquisador
          if any(compare_data.columns.isin([self.researcher])):
            # Pega quais indexs que nao foi rotulado pelo pesquisador
            mask = compare_data[self.researcher].isna()
            indexs = compare_data[mask].index.tolist()
            # print(indexs)
          else:
            # print('here')
            indexs = []
          
          # Quais exemplos precisam ser rotulados para decisao final
          mask = pd.Series(list(compare_data.index))
          self.pos_all = 0
          self.neg_all = 0
          for idx, row in compare_data.iterrows():
            row = row.value_counts()
            mask_row = row > 1
            final_decision = row[mask_row].index
            if len(final_decision) > 0:
              self.data.loc[idx, 'final_decision'] = str(int(final_decision[0]))
              mask[idx] = False
            else:
              self.data.loc[idx, 'final_decision'] = '0'
              mask[idx] = True


          self.shuffle = compare_data[mask].index
          # print(self.shuffle)

          # pega os dados que precisam ser rotulados
          match = pd.Series(list(chain(*[indexs,self.shuffle.tolist() ])))
          # self.match = match 
          match = match.value_counts()
          match_mask = match >1
          match = match[match_mask].index.sort_values()
          match = match.tolist()
          self.shuffle = self.shuffle[~self.shuffle.isin(match)]
          # print(self.shuffle)          
          # Ordena os dados para mostrar os que precisam ainda ser rotulados
          self.shuffle = [match,self.shuffle.tolist(), compare_data[mask].index.tolist()]
          self.shuffle = list(chain(*self.shuffle))

          # print(self.shuffle)

        if configure_text:
            # Creating 'text' column uniting title and abstract
            self.data.fillna(value="", inplace=True)
            self.data["text"] = self.data["title"] + ". " + self.data["abstract"]

        if sheet:
          try:
            self.data["label"] = self.data[self.researcher]
          except:
            self.data["label"] = np.nan


        if any(self.data.columns.isin(["label"])):
            self.data["label"] = self.data["label"].replace(
                {
                    "negative": "Not Included",
                    "0": "Not Included",
                    "excluded": "Not Included",
                    0: "Not Included",
                    1: "Included",
                    "1": "Included",
                    "positive": "Included",
                    "": np.nan,
                }
            )
        else:
            self.data["label"] = np.nan

        # Buttons
        self.next_b = widgets.Button(description="Next")
        self.previous_b = widgets.Button(description="Previous")
        self.Included = widgets.Button(description="Included")
        self.NIncluded = widgets.Button(description="Not Included")
        self.erase_b = widgets.Button(description="Erase")
        self.export_spreed = widgets.Button(description="Save")

        # On click functions
        self.next_b.on_click(self.next_button)
        self.previous_b.on_click(self.prev_button)
        self.Included.on_click(self.Included_button)
        self.NIncluded.on_click(self.Not_Included_button)
        self.erase_b.on_click(self.erase_button)
        self.export_spreed.on_click(self.export_spreed_button)

        # Button placement configuration
        self.Judger_hbox1 = widgets.HBox([self.NIncluded, self.Included])
        self.Judger_hbox2 = widgets.HBox([self.previous_b, self.next_b, self.erase_b])
        if sheet:
          self.Judger_grid = widgets.VBox([self.Judger_hbox1,
                                          self.Judger_hbox2,
                                          self.export_spreed])
          # self.data[self.researcher] = 
        else:
          self.Judger_grid = widgets.VBox([self.Judger_hbox1,
                                          self.Judger_hbox2])


    # Include in review button
    def Included_button(self, p):
        self.data.loc[self.shuffle[self.index], "label"] = "Included"
        try:
          self.index += 1
          self.__call__()
        except:
          self.index = 0
          self.__call__()

    # Exclude from review button
    def Not_Included_button(self, p):
        self.data.loc[self.shuffle[self.index], "label"] = "Not Included"
        try:
          self.index += 1
          self.__call__()
        except:
          self.index = 0
          self.__call__()

    # Erase label button
    def erase_button(self, p):
        text = self.data.loc[self.shuffle[self.index], "text"]
        self.data.loc[self.shuffle[self.index], "label"] = np.nan

        self.__call__()

    def export_spreed_button(self, p):
        data = self.data.copy()
        # data=data.fillna('')
        labels = data['label']
        update_rows = labels[~(labels.isna())]
        cells = []

        data = data.drop(columns=['text','label', 'title', 'abstract'])
        indexs = update_rows.index 

        if any(data.columns.isin([self.researcher])):
          column = data.columns.get_loc(self.researcher) 
          cells.append(Cell(1,column+3, self.researcher))
        else:
          column = len(data.columns) 
          cells.append(Cell(1,column+2, self.researcher))
        
        for row in indexs:
          cells.append(Cell(row+2, column + 3, update_rows[row]))

        self.sheet.update_cells(cells)
        self.__call__()
        print('\nSAVED!!')
        # print(self.update_rows)

    # Next button
    def next_button(self, p):
        # Checks if it's possible to go to the next text
        try:
            self.index += 1
            text = self.data.loc[self.shuffle[self.index], "text"]
        except:
            self.index = 0
        self.__call__()

    # Prev button
    def prev_button(self, p):
        # Checks if it's possible to go to the previous text
        if self.index - 1 < 0:
            self.index = len(self.data) - 1
        else:
            self.index -= 1

        self.__call__()

    # Call function
    def __call__(self):
        clear_output()
        # Unlabeled data
        criteria_na = self.data["label"].isna()

        # Text and label for this index
        text = self.data.loc[self.shuffle[self.index], "text"]
        try:
          label = self.data.loc[self.shuffle[self.index], self.researcher]
        except:
          label = self.data.loc[self.shuffle[self.index], "label"]

        if sheet:
            all_statistics = self.data['final_decision'].value_counts()
            try:
              statistics = self.data[self.researcher].value_counts()
            except:
              statistics = self.data["label"].value_counts()
          
        else:
          statistics = self.data["label"].value_counts()

        try:
            all_neg = all_statistics["-1"]
        except:
            all_neg = 0
        try:
            all_pos = all_statistics["1"]
        except:
            all_pos = 0
        try:
            all_desagre = all_statistics["0"]
        except:
            all_desagre = 0


        try:
            neg = statistics["Not Included"]
        except:
            neg = 0
        try:
            pos = statistics["Included"]
        except:
            pos = 0

        self.pos = pos
        self.neg = neg

        # Printing some results
        if self.sheet:
          print(f"User: {self.researcher}")
        print(f"  Current example: {self.index+1} / {len(self.data)}")
        print(f"  Unlabeled: {sum(criteria_na)}")
        print(f"  Not included examples: {neg} | Included examples: {pos}")

        display(self.Judger_grid)
        print("Text:")
        display(Markdown(text))
        print("Label:", label)
        if self.sheet:
          print(f"Index: {self.shuffle[self.index] +2}")
          print("\n")
          print("All dataset:")
          print(f"  Desagree\\Unlabeled examples: {all_desagre}")
          print(f"  Not included examples: {all_neg} | Included examples: {all_pos}")



# Labeling

To save in spreadsheets, always click on `Save` button and use the same researcher's name

In [23]:
# authenticate Google user to use spreadsheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Load spreadsheets
wd= gc.open('SLRC_LIVRO_LAMFO')
sheet = wd.worksheet('classification')
data = sheet.get_all_records()
data = pd.DataFrame(data)

# Run Classifier intaraction
judger=Judger(data, random=False,
              researcher='Nome',
              sheet=sheet)

judger()

User: Nome
  Current example: 1 / 5615
  Unlabeled: 5615
  Not included examples: 0 | Included examples: 0


Text:


Artificial Intelligence and Patient-Centered Decision-Making. 

Label: nan
Index: 6


All dataset:
  Desagree\Unlabeled examples: 5564
  Not included examples: 25 | Included examples: 26
